In [1]:
import sys

from os.path import basename
from pix2code.model.classes.Sampler import *
from pix2code.model.classes.model.pix2code import *
from pix2code.model.classes.model.pix2code_v1 import *
from pix2code.model.classes.model.pix2code_v1_Bi_GRU import *

In [2]:
# trained_weights_path = 'pix2code/bin'
trained_weights_path = 'pix2code/bin/tag_extension/6_pix2code_all_tag_early'

meta_dataset = np.load("{}/meta_dataset.npy".format(trained_weights_path), allow_pickle=True)
input_shape = meta_dataset[0]
output_size = meta_dataset[1]

model = pix2code_v1_Bi_GRU(input_shape, output_size, trained_weights_path)
model.load('pix2code')

c:\dev\anaconda3\envs\capstone\lib\site-packages\keras\optimizer_v2\rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [3]:
sampler = Sampler(trained_weights_path, input_shape, output_size, CONTEXT_LENGTH)

Vocabulary size: 23
Input shape: (256, 256, 3)
Output size: 23


In [4]:
# dataGenerator/data/png/ 폴더 파일 리스트
import os
from tqdm import tqdm
input_path = 'dataGenerator/data/training_set_tag_extension/png/'
output_path = 'dataGenerator/data/training_set_tag_extension/dsl_predict/'
file_list = os.listdir(input_path)
os.makedirs(output_path, exist_ok=True)
for input_file in tqdm(file_list):
    file_name = basename(input_file)[:basename(input_file).find(".")]
    # print(file_name)
    evaluation_img = Utils.get_preprocessed_img(input_path + input_file, IMAGE_SIZE)
    result, _ = sampler.predict_greedy(model, np.array([evaluation_img]))
    result = result.replace('<START>', '').replace('<END>', '').replace('{', ' {').replace(',', ', ')
    with open(f'{output_path}{file_name}.gui', 'w') as f:
        f.write(result)
    # print("Result greedy: {}".format(result))

100%|██████████| 8/8 [00:25<00:00,  3.19s/it]


In [5]:
%reset -f

In [6]:
import sys

from os.path import basename
from pix2code.model.classes.Sampler import *
from pix2code.model.classes.model.pix2code import *
from pix2code.model.classes.model.pix2code_v1 import *
from pix2code.model.classes.model.pix2code_v1_Bi_GRU import *
# trained_weights_path = 'pix2code/bin'
trained_weights_path = 'pix2code/bin/tag_extension/3_button_color'

meta_dataset = np.load("{}/meta_dataset.npy".format(trained_weights_path), allow_pickle=True)
input_shape = meta_dataset[0]
output_size = meta_dataset[1]

model = pix2code_v1_Bi_GRU(input_shape, output_size, trained_weights_path)
model.load('pix2code')
sampler = Sampler(trained_weights_path, input_shape, output_size, CONTEXT_LENGTH)
# dataGenerator/data/png/ 폴더 파일 리스트
import os
from tqdm import tqdm
input_path = 'dataGenerator/data/3_color/png/'
output_path = 'dataGenerator/data/3_color/dsl_predict/'
file_list = os.listdir(input_path)
os.makedirs(output_path, exist_ok=True)
for input_file in tqdm(file_list):
    file_name = basename(input_file)[:basename(input_file).find(".")]
    # print(file_name)
    evaluation_img = Utils.get_preprocessed_img(input_path + input_file, IMAGE_SIZE)
    result, _ = sampler.predict_greedy(model, np.array([evaluation_img]))
    result = result.replace('<START>', '').replace('<END>', '').replace('{', ' {').replace(',', ', ')
    with open(f'{output_path}{file_name}.gui', 'w') as f:
        f.write(result)
    # print("Result greedy: {}".format(result))

Vocabulary size: 22
Input shape: (256, 256, 3)
Output size: 22


100%|██████████| 2000/2000 [1:40:38<00:00,  3.02s/it]


In [7]:
%reset -f

In [8]:
import sys

from os.path import basename
from pix2code.model.classes.Sampler import *
from pix2code.model.classes.model.pix2code import *
from pix2code.model.classes.model.pix2code_v1 import *
from pix2code.model.classes.model.pix2code_v1_Bi_GRU import *
# trained_weights_path = 'pix2code/bin'
trained_weights_path = 'pix2code/bin/tag_extension/4_triple'

meta_dataset = np.load("{}/meta_dataset.npy".format(trained_weights_path), allow_pickle=True)
input_shape = meta_dataset[0]
output_size = meta_dataset[1]

model = pix2code_v1_Bi_GRU(input_shape, output_size, trained_weights_path)
model.load('pix2code')
sampler = Sampler(trained_weights_path, input_shape, output_size, CONTEXT_LENGTH)
# dataGenerator/data/png/ 폴더 파일 리스트
import os
from tqdm import tqdm
input_path = 'dataGenerator/data/4_triple/png/'
output_path = 'dataGenerator/data/4_triple/dsl_predict/'
file_list = os.listdir(input_path)
os.makedirs(output_path, exist_ok=True)
for input_file in tqdm(file_list):
    file_name = basename(input_file)[:basename(input_file).find(".")]
    # print(file_name)
    evaluation_img = Utils.get_preprocessed_img(input_path + input_file, IMAGE_SIZE)
    result, _ = sampler.predict_greedy(model, np.array([evaluation_img]))
    result = result.replace('<START>', '').replace('<END>', '').replace('{', ' {').replace(',', ', ')
    with open(f'{output_path}{file_name}.gui', 'w') as f:
        f.write(result)
    # print("Result greedy: {}".format(result))

Vocabulary size: 21
Input shape: (256, 256, 3)
Output size: 21


100%|██████████| 2000/2000 [1:52:25<00:00,  3.37s/it]  


In [ ]:
from multiprocessing import Queue, Process, Pool

# queue 입력 데이터
# [input_img, current_context, id]
# 마지막은 -1
inQ = Queue()
outQ = Queue()

def doPredict(model:pix2code_v1_Bi_GRU, input_imgs, current_contexts, ids, outQueue:Queue):
    result = model.predict_batch(input_imgs, current_contexts)
    output = {}
    for i in range(len(result)):
        output[ids[i]] = result[i]
    for i in range(len(result)):
        outQueue.put(output)

def worker(inQueue:Queue, outQueue:Queue, model:pix2code_v1_Bi_GRU):
    input_imgs = np.array()
    current_contexts = np.array()
    ids = []
    cnt = 0
    while True:
        data = inQueue.get()
        if data is None:
            continue
        elif data==-1:
            break
        # 마지막 데이터들 처리
        elif data==0:
            doPredict(model, input_imgs, current_contexts, ids, outQueue)
            input_imgs = np.array()
            current_contexts = np.array()
            ids = []
            cnt = 0
        if cnt == 10:
            doPredict(model, input_imgs, current_contexts, ids, outQueue)
            input_imgs = np.array()
            current_contexts = np.array()
            ids = []
            cnt = 0
        input_imgs.append(data[0])
        current_contexts.append(data[1])
        ids.append(data[2])
        cnt += 1
        
        

In [ ]:
def predict_greedy_multi(model, input_img, inQueue:Queue, outQueue:Queue, id, voc:Vocabulary, input_shape, output_size, context_length, require_sparse_label=True, sequence_length=150, verbose=False):
    current_context = [voc.vocabulary[PLACEHOLDER]] * (context_length - 1)
    current_context.append(voc.vocabulary[START_TOKEN])
    if require_sparse_label:
        current_context = Utils.sparsify(current_context, output_size)

    predictions = START_TOKEN
    out_probas = []

    for i in range(0, sequence_length):
        if verbose:
            print("predicting {}/{}...".format(i, sequence_length))

        ##################
        inQueue.put((input_img, np.array([current_context], id)))
        probases = outQueue.get()
        probas = probases[id]
        ##################
        # probas = model.predict(input_img, np.array([current_context]))
        prediction = np.argmax(probas)
        out_probas.append(probas)

        new_context = []
        for j in range(1, context_length):
            new_context.append(current_context[j])

        if require_sparse_label:
            sparse_label = np.zeros(output_size)
            sparse_label[prediction] = 1
            new_context.append(sparse_label)
        else:
            new_context.append(prediction)

        current_context = new_context

        predictions += voc.token_lookup[prediction]

        if voc.token_lookup[prediction] == END_TOKEN:
            break

    return predictions, out_probas